In [1]:
import pandas as pd
anime = pd.read_csv('anime.csv')
anime.head(50)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [3]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 480.3+ KB


In [2]:
anime.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [5]:
anime_dup = anime[anime.duplicated()]
print(anime_dup)

Empty DataFrame
Columns: [anime_id, name, genre, type, episodes, rating, members]
Index: []


In [6]:
type_values = anime['type'].value_counts()
print(type_values)

TV         3787
OVA        3311
Movie      2348
Special    1676
ONA         659
Music       488
Name: type, dtype: int64


In [4]:
m = anime['members'].quantile(0.75)
print(m)

9437.0


In [5]:
qualified_anime = anime.copy().loc[anime['members']>m]
C = anime['rating'].mean()
def WR(x,C=C, m=m):
    v = x['members']
    R = x['rating']
    return (v/(v+m)*R)+(m/(v+m)*C)
qualified_anime['score'] = WR(qualified_anime)
qualified_anime.sort_values('score', ascending =False)
qualified_anime.head(1)

,anime_id,name,genre,type,episodes,rating,members,score
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,9.239896


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Step 1: Load the data
anime = pd.read_csv('anime.csv')

# Step 2: Data Preprocessing
anime['genre'] = anime['genre'].fillna('')  # Fill missing genre values with an empty string
anime['rating'] = anime['rating'].fillna(0)  # Fill missing rating values with 0

# Step 3: Feature Selection
# Combine the 'genre' and 'rating' columns into a single feature for recommendation
anime['features'] = anime['genre'] + ' ' + anime['rating'].astype(str)

# Step 4: Calculate Item Similarity
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(anime['features'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Step 5: Get Anime Recommendations
def get_recommendations(anime_title, cosine_sim=cosine_sim, anime=anime, top_n=5):
    idx = anime.index[anime['name'] == anime_title].tolist()[0]
    similar_scores = list(enumerate(cosine_sim[idx]))
    similar_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)
    top_similar_indices = [i for i, _ in similar_scores if i != idx][:top_n]
    return anime.iloc[top_similar_indices]

# Example: Get top 5 anime recommendations for the anime with the title 'Naruto'
recommendations = get_recommendations('Naruto')
print(recommendations[['name', 'genre', 'rating']])


                                                   name  \
7867                                    Iron Virgin Jun   
1573  Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...   
1930                                  Dragon Ball Super   
4067                     Ikkitousen: Extravaganza Epoch   
3038                                       Tenjou Tenge   

                                                  genre  rating  
7867  Action, Comedy, Fantasy, Martial Arts, Super P...    4.81  
1573  Action, Comedy, Martial Arts, Shounen, Super P...    7.50  
1930  Action, Adventure, Comedy, Fantasy, Martial Ar...    7.40  
4067  Action, Ecchi, Martial Arts, School, Seinen, S...    6.81  
3038  Action, Comedy, Ecchi, Martial Arts, School, S...    7.10  


In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import linear_kernel
from IPython.display import display, HTML


anime = pd.read_csv('anime.csv')


anime['genre'] = anime['genre'].fillna('')
anime['rating'] = anime['rating'].fillna(0)
anime['features'] = anime['genre'].str.lower() + ' ' + anime['rating'].astype(str).str.replace(' ', '')

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(anime['features'])


svd = TruncatedSVD(n_components=100)
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix)


def get_recommendations(anime_title, tfidf_matrix_reduced=tfidf_matrix_reduced, anime=anime, top_n=5):
    anime_title = anime_title.lower()
    idx = anime.index[anime['name'].str.lower() == anime_title].tolist()[0]
    query_vector = tfidf_matrix_reduced[idx].reshape(1, -1)
    cosine_sim = linear_kernel(query_vector, tfidf_matrix_reduced)
    
    similar_scores = list(enumerate(cosine_sim[0]))
    similar_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)
    top_similar_indices = [i for i, _ in similar_scores if i != idx][:top_n]
    return anime.iloc[top_similar_indices][['name', 'genre', 'rating']]

# Example: Get top 5 anime recommendations for the anime with the title 'Naruto'
recommendations = get_recommendations('naruto')

# Function to create HTML table from recommendations
def create_html_table(dataframe):
    html = "<table>"
    # Header row
    html += "<tr>"
    for col in dataframe.columns:
        html += "<th>{}</th>".format(col)
    html += "</tr>"
    # Data rows
    for _, row in dataframe.iterrows():
        html += "<tr>"
        for val in row.values:
            html += "<td>{}</td>".format(val)
        html += "</tr>"
    html += "</table>"
    return html

# Display the recommendations in an HTML table in the notebook output
display(HTML(create_html_table(recommendations)))


name,genre,rating
Iron Virgin Jun,"Action, Comedy, Fantasy, Martial Arts, Super Power",4.81
Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono,"Action, Comedy, Martial Arts, Shounen, Super Power",7.5
Dragon Ball Super,"Action, Adventure, Comedy, Fantasy, Martial Arts, Shounen, Super Power",7.4
Ikkitousen: Extravaganza Epoch,"Action, Ecchi, Martial Arts, School, Seinen, Super Power",6.81
Tenjou Tenge,"Action, Comedy, Ecchi, Martial Arts, School, Shounen, Super Power",7.1
